In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -q transformers accelerate datasets pycocotools pillow tqdm



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's de

In [2]:
import os, sys, random, json, glob
from pprint import pprint
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import OwlViTProcessor, OwlViTForObjectDetection
import matplotlib.pyplot as plt

# Disable tokenizer parallelism warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("Python:", sys.version.splitlines()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


2025-08-15 07:06:04.814932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755241565.167081      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755241565.267713      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
PyTorch: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4


In [3]:
# Change these to match your Kaggle paths
REFCOCO_ANN = "/kaggle/input/refcoco/anns/refcoco/train.json"
COCO_IMG_DIR = "/kaggle/input/coco-image-caption/train2014/train2014"

print("Loading annotations from:", REFCOCO_ANN)
with open(REFCOCO_ANN, 'r') as f:
    ann = json.load(f)

print("Annotation entries:", len(ann))
print("Example entry:")
pprint(ann[0])


Loading annotations from: /kaggle/input/refcoco/anns/refcoco/train.json
Annotation entries: 42404
Example entry:
{'bbox': [103, 299, 237, 476],
 'cat': 0,
 'img_name': 'COCO_train2014_000000581857.jpg',
 'segment_id': 0,
 'sentences': [{'idx': 0, 'sent': 'the lady with the blue shirt', 'sent_id': 0},
               {'idx': 1, 'sent': 'lady with back to us', 'sent_id': 1},
               {'idx': 2, 'sent': 'blue shirt', 'sent_id': 2}],
 'sentences_num': 3}


In [4]:
def build_examples(annotation_list, images_root):
    examples = []
    for ann in annotation_list:
        img_path = os.path.join(images_root, ann['img_name'])
        bbox = ann['bbox']  # [x1, y1, x2, y2]
        for s in ann['sentences']:
            examples.append({
                "image_path": img_path,
                "text": s['sent'],
                "bbox": bbox
            })
    return examples

examples = build_examples(ann, COCO_IMG_DIR)
print(f"Total examples: {len(examples)}")
print("First example:", examples[0])

Total examples: 120624
First example: {'image_path': '/kaggle/input/coco-image-caption/train2014/train2014/COCO_train2014_000000581857.jpg', 'text': 'the lady with the blue shirt', 'bbox': [103, 299, 237, 476]}


In [5]:
def build_examples(annotation_list, images_root):
    examples = []
    for ann in annotation_list:
        img_path = os.path.join(images_root, ann['img_name'])
        bbox = ann['bbox']  # [x1, y1, x2, y2]
        for s in ann['sentences']:
            examples.append({
                "image_path": img_path,
                "text": s['sent'],
                "bbox": bbox
            })
    return examples

examples = build_examples(ann, COCO_IMG_DIR)
print(f"Total examples: {len(examples)}")
print("First example:", examples[0])


Total examples: 120624
First example: {'image_path': '/kaggle/input/coco-image-caption/train2014/train2014/COCO_train2014_000000581857.jpg', 'text': 'the lady with the blue shirt', 'bbox': [103, 299, 237, 476]}


In [6]:
random.shuffle(examples)
train_examples = examples[:int(0.8 * len(examples))]
val_examples   = examples[int(0.8 * len(examples)):]

print(f"Train: {len(train_examples)}, Val: {len(val_examples)}")


Train: 96499, Val: 24125


In [7]:
def xyxy_to_cxcywh_norm(box_xyxy, img_w, img_h):
    x1, y1, x2, y2 = box_xyxy
    w = x2 - x1
    h = y2 - y1
    cx = x1 + w / 2.0
    cy = y1 + h / 2.0
    return torch.tensor([cx/img_w, cy/img_h, w/img_w, h/img_h], dtype=torch.float32)

def cxcywh_to_xyxy(box):
    cx, cy, w, h = box.unbind(-1)
    x1 = cx - 0.5 * w
    y1 = cy - 0.5 * h
    x2 = cx + 0.5 * w
    y2 = cy + 0.5 * h
    return torch.stack([x1, y1, x2, y2], dim=-1)

def iou_cxcywh(pred, target, eps=1e-6):
    p_xy = cxcywh_to_xyxy(pred)
    t_xy = cxcywh_to_xyxy(target)
    x1 = torch.max(p_xy[...,0], t_xy[...,0])
    y1 = torch.max(p_xy[...,1], t_xy[...,1])
    x2 = torch.min(p_xy[...,2], t_xy[...,2])
    y2 = torch.min(p_xy[...,3], t_xy[...,3])
    inter = (x2 - x1).clamp(min=0) * (y2 - y1).clamp(min=0)
    area_p = (p_xy[...,2] - p_xy[...,0]).clamp(min=0) * (p_xy[...,3] - p_xy[...,1]).clamp(min=0)
    area_t = (t_xy[...,2] - t_xy[...,0]).clamp(min=0) * (t_xy[...,3] - t_xy[...,1]).clamp(min=0)
    return inter / (area_p + area_t - inter + eps)

In [8]:
class RefCOCODataset(Dataset):
    def __init__(self, examples, processor):
        self.examples = examples
        self.processor = processor

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        image = Image.open(ex["image_path"]).convert("RGB")
        text = [ex["text"]]
        img_w, img_h = image.size
        gt_box = xyxy_to_cxcywh_norm(torch.tensor(ex["bbox"], dtype=torch.float32), img_w, img_h)

        enc = self.processor(
            images=image,
            text=text,
            padding="max_length",
            max_length=16,
            truncation=True,
            return_tensors="pt"
        )

        enc = {k: v.squeeze(0) if isinstance(v, torch.Tensor) else v for k,v in enc.items()}
        enc["gt_box"] = gt_box
        return enc

def collate_fn(batch):
    return {
        "pixel_values": torch.stack([b["pixel_values"] for b in batch]),
        "input_ids": torch.stack([b["input_ids"] for b in batch]),
        "attention_mask": torch.stack([b["attention_mask"] for b in batch]),
        "gt_box": torch.stack([b["gt_box"] for b in batch])
    }

In [9]:
MODEL_ID = "google/owlvit-base-patch32"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 2
ACCUM_STEPS = 4
EPOCHS = 20  # train fully
LR = 5e-6
USE_AMP = True
save_dir = "owlvit-refcoco-finetuned"
os.makedirs(save_dir, exist_ok=True)

processor = OwlViTProcessor.from_pretrained(MODEL_ID)
model = OwlViTForObjectDetection.from_pretrained(MODEL_ID).to(DEVICE)

train_loader = DataLoader(RefCOCODataset(train_examples, processor), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(RefCOCODataset(val_examples, processor), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scaler = torch.amp.GradScaler(enabled=USE_AMP)

resume_checkpoint = os.path.join(save_dir, "checkpoint.pth")
start_epoch = 0
best_val_loss = float("inf")

if os.path.exists(resume_checkpoint):
    print(f"Resuming from {resume_checkpoint}")
    checkpoint = torch.load(resume_checkpoint, map_location=DEVICE)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    scaler.load_state_dict(checkpoint["scaler"])
    start_epoch = checkpoint["epoch"] + 1
    best_val_loss = checkpoint["best_val_loss"]
    print(f"Resumed at epoch {start_epoch}, best_val_loss={best_val_loss:.4f}")

preprocessor_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/613M [00:00<?, ?B/s]

In [ ]:
from tqdm.auto import tqdm

for epoch in range(start_epoch, EPOCHS):
    model.train()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, batch in pbar:
        batch = {k: v.to(DEVICE) if torch.is_tensor(v) else v for k,v in batch.items()}
        with torch.amp.autocast("cuda", enabled=USE_AMP):
            outputs = model(
                input_ids=batch["input_ids"],
                pixel_values=batch["pixel_values"],
                attention_mask=batch["attention_mask"]
            )

            pred_boxes = outputs.pred_boxes
            gt_box = batch["gt_box"]
            B, Q, _ = pred_boxes.shape
            gt_box_exp = gt_box.unsqueeze(1).expand(-1, Q, -1)
            ious = iou_cxcywh(pred_boxes, gt_box_exp)
            best_idx = torch.argmax(ious, dim=1)
            chosen_preds = pred_boxes[torch.arange(B), best_idx]
            chosen_ious = ious[torch.arange(B), best_idx]
            l1 = torch.nn.functional.l1_loss(chosen_preds, gt_box, reduction="none").mean(dim=1)
            loss = (l1 + (1.0 - chosen_ious)).mean()

        scaler.scale(loss / ACCUM_STEPS).backward()

        if (step + 1) % ACCUM_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        running_loss += loss.item()
        if (step + 1) % 50 == 0:
            pbar.set_description(f"epoch {epoch+1} loss {running_loss/(step+1):.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(DEVICE) if torch.is_tensor(v) else v for k,v in batch.items()}
            with torch.amp.autocast("cuda", enabled=USE_AMP):
                outputs = model(
                    input_ids=batch["input_ids"],
                    pixel_values=batch["pixel_values"],
                    attention_mask=batch["attention_mask"]
                )
                pred_boxes = outputs.pred_boxes
                gt_box = batch["gt_box"]
                B, Q, _ = pred_boxes.shape
                gt_box_exp = gt_box.unsqueeze(1).expand(-1, Q, -1)
                ious = iou_cxcywh(pred_boxes, gt_box_exp)
                best_idx = torch.argmax(ious, dim=1)
                chosen_preds = pred_boxes[torch.arange(B), best_idx]
                chosen_ious = ious[torch.arange(B), best_idx]
                l1 = torch.nn.functional.l1_loss(chosen_preds, gt_box, reduction="none").mean(dim=1)
                val_loss += (l1 + (1.0 - chosen_ious)).mean().item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1} train_loss {running_loss/len(train_loader):.4f} val_loss {val_loss:.4f}")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        model.save_pretrained(save_dir)
        processor.save_pretrained(save_dir)
        print(f"✅ Saved best model at epoch {epoch+1}")

    # Save checkpoint
    torch.save({
        "epoch": epoch,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scaler": scaler.state_dict(),
        "best_val_loss": best_val_loss
    }, resume_checkpoint)
    print(f"💾 Checkpoint saved at epoch {epoch+1}")

  0%|          | 0/48250 [00:00<?, ?it/s]